In [3]:
import numpy as np
import pandas as pd
import lightgbm as lgb

In [7]:
#read data
train_df = pd.read_csv('../data/train.csv')
meal_df = pd.read_csv('../data/meal_info.csv')
fufil_df = pd.read_csv('../data/fulfilment_center_info.csv')
test_df = pd.read_csv('../data/test_QoiMO9B.csv')

In [16]:
# concat train and test dataframes
trn_size, tst_size = train_df.shape[0], test_df.shape[0]
df = pd.concat([train_df, test_df], axis=0, sort=False)

In [17]:
# join dataframes
df = pd.merge(df, meal_df, on='meal_id', how='left')
df = pd.merge(df, fufil_df, on='center_id', how='left')

In [6]:
pd.Series(np.arange(10)).shift(1)

0    NaN
1    0.0
2    1.0
3    2.0
4    3.0
5    4.0
6    5.0
7    6.0
8    7.0
9    8.0
dtype: float64

In [13]:
# category encoding
df['category'] = df['category'].astype('category').cat.codes
df['cuisine'] = df['cuisine'].astype('category').cat.codes
df['city_code'] = df['city_code'].astype('category').cat.codes
df['center_type'] = df['center_type'].astype('category').cat.codes

In [14]:
df.center_type

0         2
1         2
2         2
3         2
4         2
5         2
6         2
7         2
8         2
9         2
10        2
11        2
12        2
13        2
14        2
15        2
16        2
17        2
18        2
19        2
20        2
21        2
22        2
23        2
24        2
25        2
26        2
27        2
28        2
29        2
         ..
489091    0
489092    0
489093    0
489094    0
489095    0
489096    0
489097    0
489098    0
489099    0
489100    0
489101    0
489102    0
489103    0
489104    0
489105    0
489106    0
489107    0
489108    0
489109    0
489110    0
489111    0
489112    0
489113    0
489114    0
489115    0
489116    0
489117    0
489118    0
489119    0
489120    0
Name: center_type, Length: 489121, dtype: int8

In [5]:
# history num_orders feature

# get the start week of meal_id
meal_start_week = df.groupby('meal_id')['week'].min().reset_index().rename(columns={'week':'start_week'})
df = pd.merge(df, meal_start_week, on='meal_id', how='left')

In [22]:
df.groupby(by=['meal_id', 'center_id', 'week'])['num_orders'].mean().groupby(level=[0]).apply(lambda x: x.cumsum() / x.size)

meal_id  center_id  week
1062     10         1        0.072671
                    2        0.138368
                    3        0.209863
                    4        0.310846
                    5        0.391330
                    6        0.483240
                    7        0.610350
                    8        0.706881
                    9        0.814585
                    10       0.938335
                    11       1.052844
                    12       1.161640
                    13       1.250189
                    14       1.304629
                    15       1.369319
                    16       1.425019
                    17       1.542972
                    18       1.606570
                    19       1.716626
                    20       1.820885
                    21       1.939931
                    22       2.049987
                    23       2.180459
                    24       2.264303
                    25       2.345879
                    26   

In [23]:
# center wise mean num_orders
center_week_orders_mean = df.groupby(['meal_id', 'center_id', 'week'])['num_orders'].mean().reset_index()
# meal wise mean num_orders
meal_week_orders_mean = df.groupby(['meal_id', 'week'])['num_orders'].mean().reset_index()

In [42]:
df.meal_id.nunique(), df.center_id.nunique()

(51, 77)

In [67]:
center_week_orders_mean_pivot = center_week_orders_mean.pivot_table(index=['meal_id', 'center_id'], columns='week', values='num_orders')

In [79]:
center_week_orders_mean_pivot.reset_index(level=[0, 1], inplace=True)

In [88]:
[-3:1]

SyntaxError: invalid syntax (<ipython-input-88-345c44f3b688>, line 1)